In [2]:
import rvnewop as rv
from glob import glob
import matplotlib.pyplot as plt
from itertools import accumulate

In [3]:
# remove st.hst, edn.hst, minver.hst, cubic.hst, wikisort.hst, nbody.hst
floating_point_hsts = list(map(lambda x: "./embench_hst/"+x, ["st.hst", "edn.hst", "minver.hst", "cubic.hst", "wikisort.hst", "nbody.hst"]))
for FILENAME in [x for x in glob("./embench_hst/*.hst") if x not in floating_point_hsts]:
    #print("Processing {}...".format(FILENAME))

    prog = rv.Histogram.parse(FILENAME, isa="32ICM")
    prog.findBasicBlocks()

    prog.addLivenessValuesToGraph()

    # total cycles = sum of all the frequencies
    total_cycles = sum(
        [inst.freq for bb in prog.basicBlocks for inst in bb.bbInstructions()]
    )

    new_instructions = []

    for i, b in enumerate((prog.getSubBlocks())):
        graph = b.constructDAG()

        # temp will store every permutation of candidate subgraphs
        temp = []
        for n in rv.analysis.findCandidateSubgraphs(prog, graph):
            subtree = rv.analysis.createSubtreeFromNode(graph, n)
            root = n

            temp.append(rv.Subgraph(subtree, root))

        # add candidate subgraph with highest score to new instructions list
        current_nodes = set()
        temp = sorted(temp, key=lambda sg: sg.score, reverse=True)
        while temp:
            if any([(n in current_nodes) for n in temp[0].graph.nodes]):
                temp.pop(0)
            else:
                new_instructions.append(temp.pop(0))

    new_instructions = sorted(new_instructions, key=lambda sg: sg.score, reverse=True)

    inst_dict = {}
    for subgraph in new_instructions:
        key = rv.analysis.graphToParenString(subgraph.graph)

        if key in inst_dict:
            inst_dict[key].score += subgraph.score
        else:
            inst_dict[key] = subgraph

    new_instructions = [inst_dict[key] for key in inst_dict]

    max_depth = 0
    max_mult_depth = 0
    for sg in new_instructions:
        if sg.containsMultiplyInstruction():
            max_mult_depth = max(max_mult_depth, sg.depth)
        else:
            max_depth = max(max_depth, sg.depth)

    saved_cycles = sum([sg.score for sg in new_instructions])
    percent_cycles = float(saved_cycles) / total_cycles * 100
    #plt.clf()
    #plt.plot([x for x in range(len(new_instructions))],list(map(lambda x:x/total_cycles, accumulate([sg.score for sg in new_instructions]))))
    #plt.show()
    #print(
    #    """Saved {} cycles out of {} ({:.4}%) with {} new instructions""".format(
    #        saved_cycles, total_cycles, percent_cycles, len(new_instructions)
    #    )
    #)
    #print("max depth: {}\nmax mult depth: {}".format(max_depth, max_mult_depth))
    
    print("{},{:.4},{:.4},{:.4},{:.4},{:.4}".format(FILENAME[len("./embench_hst/"):-4], *[float(x.score) / total_cycles for x in new_instructions[:5]]))


aha-mont64,0.01163,0.01245,0.0117,0.0338,0.003876
ud,0.04087,0.1033,0.03065,0.06885,0.005659
slre,0.03372,0.0108,0.002422,0.003903,0.002112
qrduino,0.03937,0.01461,0.01218,0.02953,0.02606
matmult-int,0.3966,0.3173,0.238,0.0008095,0.0003334
picojpeg,0.01623,0.01673,0.02644,0.02266,0.01889
sglib-combined,0.01099,0.0493,0.006591,0.04353,0.00226
nsichneu,0.003729,0.001492,0.0007458,0.0007458,0.0005007
coremark,0.04343,0.008624,0.008314,0.01364,0.01393
nettle-sha256,0.00464,0.002368,0.00232,0.004799,0.05568
huffbench,0.08494,0.03322,0.1554,0.02204,0.02058
crc32,0.2017,0.1765,0.1261,0.1009,0.07566
nettle-aes,0.1327,0.06633,0.03596,0.002073,0.008292
statemate,0.007563,0.003025,0.001915,0.001513,0.001513
